In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [3]:
class FordFulkerson:
    def __init__(self, N: int):
        self.N = N
        # 残余グラフを準備
        self.G = [list() for i in range(N+1)]
        
    # 残余グラフとなる重み付き隣接頂点リストを初期化、容量 cap の辺 (u: fr, v: to) がある
    def add_edge(self, fr: int, to: int, cap):
        # forward: 頂点 fr → to に向けて増やせるフロー (初期値: cap) を格納
        forward = [to, cap, None]
        # backward: 頂点 to → fr に向けて戻せるフロー (初期値 0) を格納
        # さらに、forward[2] = backward, backward[2] = forword として、
        # 逆方向に対応する辺を取得できるようにしておく
        forward[2] = backward = [fr, 0, forward]
        # 残余グラフへ、forward, backward を格納
        self.G[fr].append(forward)
        self.G[to].append(backward)
    
    # 深さ優先探索
    # pos: 探索のスタート地点, goal: 出口の頂点, f: 流せるフローの最大値
    def dfs(self, pos, goal, f):
        if pos == goal:
            # ゴールに到着できる経路を一つ見つけた、流量 f のフローを流せる
            return f
        # visited: 探索済みフラグ
        visited = self.visited
        visited[pos] = True
        for e in self.G[pos]:
            # nex: 次に探索のスタート地点とする pos の隣接頂点, rev: 逆方向に対応する辺
            nex, cap, rev = e
            # 容量 cap が 1 以上でかつ、まだ探索していない頂点を探索
            if cap > 0 and not visited[nex]:
                # flow: ゴールに到着して初めて、f >= 1 となる値が返される (再帰処理)
                # f = min(min(f, cap)) でゴールに到着する経路内に流せる最大フローを更新
                flow = self.dfs(nex, goal, min(f, cap))
                # 経路内の辺に対応する残余グラフを更新していく
                if flow >= 1:
                    # 増やせるフローを減算
                    e[1] -= flow
                    # 戻せるフローを加算
                    rev[1] += flow
                    # 再帰元の関数に flow を返して伝搬
                    return flow
                
        # 全ての辺を探索したが、ゴールまでのフローが存在しない
        return 0
    
    # 頂点 s から頂点 t までの最大フローの総流量を返す
    def maxflow(self, s, t):
        total_flow = 0
        f = INF = 10**15
        N = self.N
        # ゴールとなる頂点 t までフローを流せる経路が見つからなくなったら終了
        while f:
            self.visited = [False] * (N+1)
            f = self.dfs(s, t, INF)
            total_flow += f
        return total_flow

N,M=map(int,input().split())
E=[list(map(int,input().split())) for _ in range(M)]

ff=FordFulkerson(N)
for u,v,c in E:
    ff.add_edge(u, v, c)

ans=ff.maxflow(1, N)
print(ans)

8


In [65]:
E

[[1, 2, 5], [1, 4, 4], [2, 3, 4], [2, 5, 7], [3, 6, 3], [4, 5, 3], [5, 6, 5]]

In [66]:
ff.G

[[],
 [[2, 5, [1, 0, [...]]], [4, 4, [1, 0, [...]]]],
 [[1, 0, [2, 5, [...]]], [3, 4, [2, 0, [...]]], [5, 7, [2, 0, [...]]]],
 [[2, 0, [3, 4, [...]]], [6, 3, [3, 0, [...]]]],
 [[1, 0, [4, 4, [...]]], [5, 3, [4, 0, [...]]]],
 [[2, 0, [5, 7, [...]]], [4, 0, [5, 3, [...]]], [6, 5, [5, 0, [...]]]],
 [[3, 0, [6, 3, [...]]], [5, 0, [6, 5, [...]]]]]